<a href="https://colab.research.google.com/github/rohbot25/Erohan_Repository/blob/main/FullSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install taichi==1.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import numpy as np
import os
import math

max_steps = 300
dt = 0.001
learning_rate = 0.0001
gravity = -9.8
soil_damping = .99 # add soil damping / spring damping
damping = 0.0
stiffness = 1000.0
ground_height = 0.1
sim_size = 512
epsilon = 0.00001

scale = 0.75
soil_particles = 150

real = ti.f32

startingObjectPositions = []

object_radius = 8
object_mass = 100
#n_objects = 4

#for objectIndex in range(n_objects):

#startingObjectPositions.append( [np.random.random(), np.random.random()* 0.9 + 0.1] )

#startingObjectPositions.append( [0.1 * scale,ground_height+0.2 ])
#startingObjectPositions.append( [0.1 * scale,ground_height+0.4 * scale])

#startingObjectPositions.append( [0.3 * scale,ground_height+0.4 * scale])

#startingObjectPositions.append( [0.3 * scale,ground_height+0.2 ])

#startingObjectPositions.append( [0.25 * scale,ground_height+0.2])
#startingObjectPositions.append( [0.25 * scale,ground_height+0.35 * scale])

#startingObjectPositions.append( [0.15 * scale,ground_height+0.35 * scale])
#startingObjectPositions.append( [0.15 * scale,ground_height+0.2])

startingObjectPositions.append( [0.1,ground_height+0.15])
startingObjectPositions.append( [0.1,ground_height+0.25])

startingObjectPositions.append( [0.2,ground_height+0.15])
startingObjectPositions.append( [0.2,ground_height+0.25])

n_objects = len(startingObjectPositions)

springs = []

def add_spring(objA,objB,motorized):
  objAIndex = objA
  objAPos = startingObjectPositions[objAIndex]
  objBIndex = objB
  objBPos = startingObjectPositions[objBIndex]
  xDistAB = objAPos[0] - objBPos[0]
  yDistAB = objAPos[1] - objBPos[1]
  distAB = math.sqrt( xDistAB**2 + yDistAB**2 )
  restingLength = distAB
  springs.append( [objAIndex, objBIndex, restingLength,motorized] )

add_spring(0,1,1)
add_spring(1,2,1)
add_spring(2,3,1)
add_spring(3,0,1)
add_spring(0,2,1)
add_spring(1,3,1)


#add_spring(0,1,1)
#add_spring(1,2,1)
#add_spring(2,3,1)
#add_spring(3,4,1)
#add_spring(4,5,1)
#add_spring(5,6,1)
#add_spring(6,7,1)
#add_spring(7,0,1)

#add_spring(0,6,1)
#add_spring(1,7,1)
#add_spring(2,4,1)
#add_spring(3,5,1)
#add_spring(1,5,1)
#add_spring(2,6,1)
#add_spring(1,6,1)
#add_spring(2,5,1)

n_springs = len ( springs )

soil = []
mass = []
for i in range(soil_particles):
  soil.append([np.random.uniform(),np.random.uniform()* 0.1 + 0.1001])
  mass.append(int(np.random.random() * 7) + 5)

#soil.append([0.5,0.5])
#soil.append([0.45,0.5])
#mass.append(5)
#mass.append(5)

ti.init( default_fp = real )

vec = lambda: ti.Vector.field(2, dtype = real)

soilPositions = vec()
soilVelocities = vec()
masses = ti.field(ti.i32)

spring_anchor_a = ti.field(ti.i32)
spring_anchor_b = ti.field(ti.i32)
spring_resting_lengths = ti.field(ti.f32)
spring_actuation = ti.field(ti.i32)

positions = vec()
velocities = vec()
springRestoringForces = vec()
springForcesOnObjects = vec()
weightsSH = ti.field(ti.f32)

n_sin_waves = 10
n_hidden_neurons = 32

center = vec()
goal = vec()

hidden = ti.field(ti.f32)
biasH = ti.field(ti.f32)

def n_sensors():
  return n_sin_waves + 4 * n_objects

weightsHM = ti.field(ti.f32)

actuation = ti.field(ti.f32)

# Declare fields for variables to debug
#normals = vec()
#tangents = vec()
#v1ns = ti.field(dtype=real)
#v1ts = ti.field(dtype=real)
#v2ns = ti.field(dtype=real)
#v2ts = ti.field(dtype=real)
#v1n2s = ti.field(dtype=real)
#v2n2s = ti.field(dtype=real)
#vec1ns = vec()
#vec1ts = vec()
#vec2ns = vec()
#vec2ts = vec()
#overlaps = ti.field(dtype=real)

#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(normals)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(tangents)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1n2s)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2n2s)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(overlaps)

ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions.grad)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(velocities)
ti.root.dense(ti.i, n_springs).place(spring_anchor_a,spring_anchor_b,spring_resting_lengths,spring_actuation)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_springs).place(springRestoringForces)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(springForcesOnObjects)
ti.root.dense(ti.ij, (n_hidden_neurons, n_sensors())).place(weightsSH)
ti.root.dense(ti.i, max_steps).place(center)
ti.root.place(goal)
ti.root.dense(ti.ij, (max_steps,n_hidden_neurons)).place(hidden)
ti.root.dense(ti.i, n_hidden_neurons).place(biasH)
ti.root.dense(ti.ij, (n_springs, n_hidden_neurons)).place(weightsHM)
ti.root.dense(ti.ij, (max_steps,n_springs)).place(actuation)

loss = ti.field( dtype=ti.f32, shape = ())

ti.root.lazy_grad()

ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilPositions)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilVelocities)
ti.root.dense(ti.i, soil_particles).place(masses)

@ti.kernel
def Compute_Loss():
  loss[None] -= positions[max_steps-1,0][0]

@ti.kernel
def Calculate_Center(timeStep: ti.i32):
  for _ in range(1):
    c = ti.Vector( [0.0, 0.0])

    for i in range( n_objects):

      c+= positions[timeStep, i]

    center[timeStep] = c / n_objects
# ------------------------------------------

def Draw(frameOffset):
  for timeStep in range(0,max_steps):

    gui = ti.GUI("Robot" , (512,512), background_color = 0xFFFFFF, show_gui=False)

    #draw the floor
    gui.line( begin = ( 0, ground_height) ,
              end = (1,ground_height) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (0,0) ,
              end = (0,1) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (1,0) ,
              end = (1,1) ,
              color = 0x0,
              radius = 1)
    #draw the soil
    for particle in range(soil_particles):
      x = soilPositions[timeStep,particle][0]
      y = soilPositions[timeStep,particle][1]
      gui.circle ( (x,y), color = 0x0 , radius = masses[particle] )
    #draw objects
    for objectIndex in range(n_objects):

      x = positions[timeStep,objectIndex][0]
      y = positions[timeStep,objectIndex][1]

      gui.circle ( (x,y), color = 0x0 , radius = object_radius )

    #draw the springs
    for springIndex in range(n_springs):
      objectAIndex = spring_anchor_a[springIndex]
      objectBIndex = spring_anchor_b[springIndex]
      posA = positions[timeStep,objectAIndex]
      posB = positions[timeStep,objectBIndex]
      motorInside = springs[springIndex][3]
      if motorInside:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 3)
      else:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 1)

    gui.show( 'test' + str(frameOffset + timeStep) + '.png')

# ----------------------------------------
def Initialize():

  for objectIndex in range(soil_particles):

    soilPositions[0,objectIndex] = soil[objectIndex]

    soilVelocities[0,objectIndex] = [0.0,0.0]

    masses[objectIndex] = mass[objectIndex]

  for objectIndex in range(n_objects):

    positions[0,objectIndex] = startingObjectPositions[objectIndex]

    velocities[0,objectIndex] = [0,-0.1]

  for springIndex in range(n_springs):
    s = springs[springIndex]
    spring_anchor_a[springIndex]        = s[0]
    spring_anchor_b[springIndex]        = s[1]
    spring_resting_lengths[springIndex] = s[2]
    spring_actuation[springIndex]       = s[3]

  for i in range(1,max_steps):
    for j in range(n_objects):
      positions[i,j][0] = 0.0
      positions[i,j][1] = 0.0
      velocities[i,j][0] = 0.0
      velocities[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_springs):
      springRestoringForces[i,j][0] = 0.0
      springRestoringForces[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_objects):
      springForcesOnObjects[i,j][0] = 0.0
      springForcesOnObjects[i,j][1] = 0.0

  for i in range(max_steps):
    for j in range(n_hidden_neurons):
      hidden[i,j] = 0.0

  for i in range(max_steps):
    for j in range(n_springs):
      actuation[i,j] = 0.0

  goal[None] = [0.9 , 0.2]

def Initialize_Neural_Network_Parameters():

  for i in range(n_hidden_neurons):
    for j in range( n_sensors()):
      weightsSH[i,j] = np.random.randn() * 0.2 - 0.1

  for i in range(n_hidden_neurons):
    biasH[i] = np.random.randn() * 2 - 1


  for i in range(n_springs):
    for j in range( n_hidden_neurons):
      weightsHM[i,j] = np.random.randn() * 0.2 - 0.1

@ti.kernel
def Simulate_Neural_Network_HM(timeStep : ti.i32):

  for i in range(n_springs):

    activation = 0.0

    for j in ti.static(range( n_hidden_neurons)):

      activation += weightsHM[i,j] * hidden[timeStep,j]

    activation = ti.tanh(activation)

    actuation[timeStep,i] = activation


@ti.kernel
def Simulate_Neural_Network_SH(timeStep : ti.i32):

  for i in range( n_hidden_neurons ):

      activation = 0.0
      for j in ti.static(range( n_sin_waves)):

        activation += weightsSH[i,j] * ti.sin(10 *  timeStep * dt + \
                                              2 * math.pi / n_sin_waves * j)
      for j in ti.static(range( n_objects )):
        offset = positions[timeStep,j] - center[timeStep]

        activation += 0.1 *  weightsSH[i,j *4     + n_sin_waves] * offset[0]
        activation += 0.1 *  weightsSH[i,j +4 + 1 + n_sin_waves] * offset[1]

        activation += 0.1 *  weightsSH[i,j *4 + 2 + n_sin_waves] * positions[timeStep,j][0]
        activation += 0.1 *  weightsSH[i,j +4 + 3 + n_sin_waves] * positions[timeStep,j][1]

      activation += 0.1 * weightsSH[i,n_objects * 4 + n_sin_waves] * \
      (goal[None][0] - center[timeStep][0])

      activation += 0.1 * weightsSH[i,n_objects * 4 + n_sin_waves + 1] * \
      (goal[None][1] - center[timeStep][1])

      activation += biasH[i]

      activation = ti.tanh(activation )

      hidden[timeStep,i] = activation

@ti.kernel
def Simulate_Springs(timeStep : ti.i32):

  for springIndex in range(n_springs):
    objectAIndex = spring_anchor_a[springIndex]
    objectBIndex = spring_anchor_b[springIndex]
    posA = positions[timeStep-1,objectAIndex]
    posB = positions[timeStep-1,objectBIndex]
    dist = posA - posB
    length = dist.norm()
    springRestingLength = spring_resting_lengths[springIndex]

    springRestingLength = springRestingLength + \
    0.02 * spring_actuation[springIndex] * \
    actuation[timeStep,springIndex]
    #ti.sin(0.5*timeStep)

    springUnhappiness = length - springRestingLength
    springRestoringForces[timeStep,springIndex] = \
    (dt * springUnhappiness * stiffness / length) * dist

    springForcesOnObjects[timeStep,objectAIndex] += -5.0*springRestoringForces[timeStep,springIndex]
    springForcesOnObjects[timeStep,objectBIndex] += 5.0*springRestoringForces[timeStep,springIndex]

# ----------------------------------------
def Simulate():

  for timeStep in range(1,max_steps):
    Simulate_Soil(timeStep)
    Step_One(timeStep)
#----------------------------------------

#@ti.kernel
@ti.ad.no_grad
def Simulate_Soil(timeStep: ti.i32):
    for objA in range(soil_particles):

      oldPosition = soilPositions[timeStep-1, objA]

      newVelocity = soilVelocities[timeStep-1, objA] + \
      dt * gravity * ti.Vector([0,1])

      newPosition = oldPosition + dt * newVelocity
      collision = False
      for objB in range(soil_particles):
        if (objB != objA):
          posA = soilPositions[timeStep-1,objA]
          posB = soilPositions[timeStep-1,objB]
          dist = posA - posB
          length = dist.norm()

          radiusA = masses[objA] / sim_size
          radiusB = masses[objB] / sim_size
          radiiSum = radiusA + radiusB
          if (length < radiiSum):
            collision = True
            if length < epsilon:
              length = epsilon

            normal = dist / length
            tangent = ti.Vector([-normal[1], normal[0]])

            soil_A_Unit_Norm = ti.Vector.dot(newVelocity, normal)
            soil_A_Unit_Tan = ti.Vector.dot(newVelocity, tangent)
            soil_B_Unit_Norm = ti.Vector.dot(soilVelocities[timeStep-1,objB], normal)
            soil_B_Unit_Tan = ti.Vector.dot(soilVelocities[timeStep-1,objB], tangent)

            soil_A_New_Norm = (soil_A_Unit_Norm * (masses[objA] - masses[objB]) + 2 * masses[objB] * soil_B_Unit_Norm) / (masses[objA] + masses[objB])
            soil_B_New_Norm = (soil_B_Unit_Norm * (masses[objB] - masses[objA]) + 2 * masses[objA] * soil_A_Unit_Norm) / (masses[objA] + masses[objB])

            vec_A_Norm = soil_A_New_Norm * normal
            vec_A_Tan = soil_A_Unit_Tan * tangent
            vec_B_Norm = soil_B_New_Norm * normal
            vec_B_Tan = soil_B_Unit_Tan * tangent

            soilVelocities[timeStep,objA] = (vec_A_Norm + vec_A_Tan) * (1-soil_damping)
            soilVelocities[timeStep,objB] = (vec_B_Norm + vec_B_Tan) * (1-soil_damping)
            overlap = (radiiSum - length) / 2
            newPosition = soilPositions[timeStep-1,objA] + overlap * normal
            soilPositions[timeStep, objB] = soilPositions[timeStep-1,objB] - overlap * normal

            while length < radiiSum:
              newPosition += overlap * normal
              soilPositions[timeStep, objB] -= overlap * normal
              length += overlap * 2

      particle_radius = masses[objA] / sim_size
      if newPosition[0] > 1 - particle_radius:
            newPosition[0] = 1 - particle_radius
            newVelocity[0] *= -1 * (1-soil_damping)

      if newPosition[0] < particle_radius:
            newPosition[0] = particle_radius
            newVelocity[0] *= -1 * (1-soil_damping)

      if newPosition[1] < ground_height + particle_radius:
            newPosition[1] = ground_height + particle_radius
            newVelocity[1] *= -1 * (1-soil_damping)

      if not collision:
        soilVelocities[timeStep,objA] = newVelocity
        soilPositions[timeStep,objA] = newPosition



@ti.kernel
def Simulate_Objects(timeStep: ti.i32):

  for robot in range(n_objects):

    velocity = velocities[timeStep-1,robot]
    position = positions[timeStep-1,robot]

    for soil in range(soil_particles):
      #collision between robot - soil particles
      posB = position
      posA = soilPositions[timeStep-1,soil]
      dist = posA - posB
      length = dist.norm()

      radiusA = masses[soil] / sim_size
      radiusB = object_radius / sim_size
      radiiSum = radiusA + radiusB
      if (length < radiiSum):
        if length < epsilon:
          length = epsilon

        normal = dist / length
        tangent = ti.Vector([-normal[1], normal[0]])

        obj_unit_norm = ti.Vector.dot(velocity, normal)
        obj_unit_tan = ti.Vector.dot(velocity, tangent)
        soil_unit_norm = ti.Vector.dot(soilVelocities[timeStep-1,soil], normal)
        soil_unit_tan = ti.Vector.dot(soilVelocities[timeStep-1,soil], tangent)


        soil_new_norm = (soil_unit_norm * (masses[soil] - object_mass) + 2 * object_radius * obj_unit_norm) / (masses[soil] + object_mass)
        obj_new_norm = (obj_unit_norm * (object_mass - masses[soil]) + 2 * masses[soil] * soil_unit_norm) / (masses[soil] + object_mass)

        vec_soil_norm = soil_new_norm * normal
        vec_soil_tan = soil_unit_tan * tangent
        vec_obj_norm = obj_new_norm * normal
        vec_obj_tan = obj_unit_tan * tangent

        soilVelocities[timeStep,soil] = (vec_soil_norm + vec_soil_tan) * (1-soil_damping)
        velocity = (vec_obj_norm + vec_obj_tan)
        overlap = (radiiSum - length) / 2
        soilPositions[timeStep,soil] = soilPositions[timeStep-1,soil] + overlap * normal
        position = positions[timeStep-1,robot] - overlap * normal

        while length < radiiSum:
          soilPositions[timeStep,soil] += overlap * normal
          position -= overlap * normal
          length += overlap * 2

    oldPosition = position

    oldVelocity = velocity + 5 * ti.Vector([0,1]) * dt + dt * gravity * ti.Vector([0,1]) + \
    springForcesOnObjects[timeStep,robot]

    newPosition = oldPosition + dt * oldVelocity

    newVelocity = oldVelocity

    if newPosition[1] < ground_height + object_radius/sim_size:
            newPosition[1] = ground_height + object_radius/sim_size
            newVelocity[1] *= -1

    positions[timeStep,robot] = newPosition

    velocities[timeStep,robot] = newVelocity

def Step_One(timeStep: ti.i32):

  Calculate_Center(timeStep)
  Simulate_Neural_Network_SH(timeStep)
  Simulate_Neural_Network_HM(timeStep)
  Simulate_Springs(timeStep)
  Simulate_Objects(timeStep)


# ----------------------------------------

def Make_Movie(frame_rate):
    os.system("rm movie.mp4")
    os.system(f"ffmpeg -r {frame_rate} -i test%d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p movie.mp4")


# ----------------------------------------
def Run_Simulation():
  Initialize()

  with ti.ad.Tape(loss):

    Simulate()
    loss[None] = 0
    Compute_Loss()

  print(loss[None])

def Tune_HM_Weights():

  for i in range(n_springs):
    for j in range(n_hidden_neurons):
      weightsHM[i,j] -= learning_rate * weightsHM.grad[i,j]

def Tune_Hidden_Layer_Biases():
  for i in range(n_hidden_neurons):
    biasH[i] -= learning_rate * biasH.grad[i]

def Tune_SH_Weights():
  #tune the weights of the synapses that connect the sensors
  #to the hidden neurons
  for i in range( n_hidden_neurons):

      activation = 0.0
      for j in ti.static(range( n_sin_waves)):

        weightsSH[i,j] -= learning_rate * weightsSH.grad[i,j]

      for j in ti.static(range( n_objects )):

        weightsSH[i,j *4     + n_sin_waves] -= learning_rate * weightsSH.grad[i,j *4     + n_sin_waves]
        weightsSH[i,j +4 + 1 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j +4 + 1 + n_sin_waves]

        weightsSH[i,j *4 + 2 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j *4 + 2 + n_sin_waves]
        weightsSH[i,j +4 + 3 + n_sin_waves] -= learning_rate * weightsSH.grad[i,j +4 + 3 + n_sin_waves]

      weightsSH[i,n_objects * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i,n_objects * 4 + n_sin_waves]

      weightsSH[i,n_objects * 4 + n_sin_waves + 1] -= learning_rate * weightsSH.grad[i,n_objects * 4 + n_sin_waves + 1]

def Tune_Robots_Brain():

  Tune_HM_Weights()

  Tune_Hidden_Layer_Biases()

  Tune_SH_Weights()

# ---------- Main body of code
#Initialize_Neural_Network_Parameters()

#for optimizationStep in range(2):
  #print(f"Optimization Step: {optimizationStep}")
  #Run_Simulation()
Initialize()
Simulate()

  #if(optimizationStep == 0):
    #os.system("rm *.png")
    #Draw(0)

  #Tune_Robots_Brain()

Draw(0)

Make_Movie(75)



# Debug printing function
def print_debug_info():
    for timeStep in range(1, max_steps):
        for objA in range(soil_particles):
            print(f"Time Step: {timeStep}, Object: {objA}")
            print(f"Normal: {normals[timeStep, objA]}")
            print(f"Tangent: {tangents[timeStep, objA]}")
            print(f"v1n: {v1ns[timeStep, objA]}")
            print(f"v1t: {v1ts[timeStep, objA]}")
            print(f"v2n: {v2ns[timeStep, objA]}")
            print(f"v2t: {v2ts[timeStep, objA]}")
            print(f"v1n2: {v1n2s[timeStep, objA]}")
            print(f"v2n2: {v2n2s[timeStep, objA]}")
            print(f"vec1n: {vec1ns[timeStep, objA]}")
            print(f"vec1t: {vec1ts[timeStep, objA]}")
            print(f"vec2n: {vec2ns[timeStep, objA]}")
            print(f"vec2t: {vec2ts[timeStep, objA]}")
            print(f"Overlap: {overlaps[timeStep, objA]}")
            print(f"Updated Position: {soilPositions[timeStep, objA]}")
            print(f"Updated Velocity: {soilVelocities[timeStep, objA]}")
            print("------------------")


#print_debug_info()

#watch movie
from IPython.display import HTML
from base64 import b64encode
mp4 = open('movie.mp4', 'rb').read()
data_url = "data:video/mp4;base64,"+ b64encode(mp4).decode()

HTML('<video width=sim_size controls> <source src="%s" type="video/mp4"></video>' % data_url)



[Taichi] Starting on arch=x64
